# Overview: `tests/test_layers.py`

This notebook cell contains **unit tests** for each core layer in our “from scratch” CNN implementation. It verifies both **forward** and **backward** behavior on small, controlled inputs.

---

## Contents

1. **ReLU**
   - **Forward**: negative inputs should become zero.
   - **Backward**: gradient only flows where input was positive.

2. **MaxPool2D**
   - **Forward**: on a 4×4 example, checks that each 2×2 window yields the correct maximum.
   - **Backward**: with `dA=1`, ensures gradient is routed back exactly to the positions of those maxima.

3. **Conv2D (identity filter)**
   - Builds a 2×2 identity filter on a 3×3×1 input.
   - **Forward**: output entries equal the sum of the corresponding diagonal input elements.
   - (Optional) **Backward**: could be tested by checking shapes and a known gradient.

4. **Flatten + Dense (softmax)**
   - **Flatten**: converts a (1,2,2,1) tensor into shape (1,4).
   - **Dense**: with a fixed seed, 
     - checks that the softmax outputs sum to 1 along each row,
     - confirms `backward` returns a gradient of shape `(1,4)`.

---

By running these tests on trivial inputs, we gain confidence that each layer’s implementation is correct before moving on to full‑scale training.

In [1]:
import sys 
from pathlib import Path
import numpy as np

cwd = Path.cwd()
if  (cwd / 'src').exists():              
    src_dir = cwd / 'src'
elif(cwd.parent / 'src').exists():        
    src_dir = cwd.parent / 'src'
else:
    raise FileNotFoundError("No pude encontrar la carpeta 'src/'")

sys.path.insert(0, str(src_dir))

from layers.conv2d      import Conv2D
from layers.activations import ReLU
from layers.pooling     import MaxPool2D
from layers.dense       import *

from train import *


### Test Relu forward

In [8]:
def test_relu_forward_backward():
    Z = np.array([[-1, 2], [0, -3]], dtype=float)
    relu = ReLU()
    A = relu.forward(Z)

    assert np.array_equal(A, np.array([[0,2],[0,0]]))
    dA = np.array([[1,1],[1,1]], dtype=float)
    dZ = relu.backward(dA)

    assert np.array_equal(dZ, np.array([[0,1],[0,0]]))

try:
    test_relu_forward_backward()
    print("✅ ReLU forward test passed!")
except AssertionError as e:
    print("❌ ReLU test failed:", e)


✅ ReLU forward test passed!


### Test Pool layer forward 

In [7]:
def test_pool_forward_backward():
    A_prev = np.arange(16).reshape(1,4,4,1).astype(float)
    pool = MaxPool2D(f=2, stride=2)
    A, cache = pool.forward(A_prev), pool.cache

    assert A.shape == (1,2,2,1)
    assert A[0,:,:,0].tolist() == [[5,7],[13,15]]


    dA = np.ones_like(A)
    dA_prev = pool.backward(dA)

    expected = np.zeros_like(A_prev)
    for val in (5,7,13,15):
        idx = np.where(A_prev==val)
        expected[idx] = 1
    assert np.array_equal(dA_prev, expected)


try:
    test_pool_forward_backward()
    print("✅ Pool layer forward test passed!")
except AssertionError as e:
    print("❌ Pool layer test failed:", e)

✅ Pool layer forward test passed!


### Test Conv Layer 

In [9]:
def test_conv_single_identity():

    A_prev = np.arange(9).reshape(1,3,3,1).astype(float)
    conv = Conv2D(n_C_prev=1, n_C=1, f=2, stride=1, pad=0, initialization='rand')


    conv.W[:] = np.array([[[[1.]] , [[0.]]],[[[0.]] , [[1.]]]])
    conv.b[:] = 0
    Z = conv.forward(A_prev)

    assert Z.shape == (1,2,2,1)
    assert Z[0,0,0,0] == A_prev[0,0,0,0] + A_prev[0,1,1,0]


try:
    test_conv_single_identity()
    print("✅ Conv2d layer forward test passed!")
except AssertionError as e:
    print("❌ Conv2d layer test failed:", e)



✅ Conv2d layer forward test passed!


### Test dense layer 

In [13]:
def test_flatten_and_dense_smoke():
    A_prev = np.array([[[[1],[2]],[[3],[4]]]], dtype=float)  # (1,2,2,1)
    flat = Flatten()
    D = flat.forward(A_prev)

    assert D.shape == (1,4)
    dense = Dense(n_units=3, initialization='rand', seed=0)
    A = dense.forward(D)

    assert np.allclose(A.sum(axis=1), 1.0)

    dA = np.random.randn(1,3)
    dD = dense.backward(dA)
    assert dD.shape == D.shape

try:
    test_flatten_and_dense_smoke()
    print("✅ Dense layer forward test passed!")
except AssertionError as e:
    print("❌ Dense layer test failed:", e)

✅ Dense layer forward test passed!


## Test the full model 

In [3]:
def test_model_pipeline_shapes():

    model = crear_modelo(filters=[3,4], pool=[1], n_classes=2)
    X = np.random.randn(2, 8, 8, 3).astype(float)  
    y = np.zeros((2,2))
    y[np.arange(2), np.random.randint(0,2,2)] = 1

    A_out, _ = conv_net_forward(model, X)
    assert A_out.shape == (2,2)


    grads = conv_net_backward(model, A_out, y, lr=1e-3)

    assert 'dW1 conv2d' in grads
    assert 'db1 conv2d' in grads


try:
    test_model_pipeline_shapes()
    print("✅ Model forward and backward test passed!")
except AssertionError as e:
    print("❌ Model forward and backward test failed:", e)

✅ Model forward and backward test passed!


## Pablo Reyes 